# Stack Overflow Analytics dengan PySpark dan NLP

**UAS - Big Data Predictive Analytics Lanjut**

**Muhammad Raihan Alfarizi - 23.11.5548**

Notebook ini menganalisis data **Stack Overflow** secara langsung menggunakan:
- Apache Spark (PySpark) untuk Big Data Processing
- NLTK untuk Natural Language Processing
- Spark MLlib untuk Machine Learning

Dataset diambil langsung dari **Stack Exchange API** (data real-time).

---

## Persyaratan UAS yang Tercakup:
1. Dataset 5V (Volume, Variety, Veracity, Value)
2. File System (Parquet storage)
3. Batch Processing + MapReduce (RDD operations)
4. EDA + Visualisasi
5. Preprocessing data
6. Spark SQL (CTE, subquery, hint)
7. RDD Operations (reduceByKey, groupByKey, combineByKey, aggregateByKey)
8. ML Komparasi 2 Algoritma
9. Hyperparameter Tuning
10. Evaluasi Model (Accuracy, F1, Precision, Recall, AUC-ROC)

## 1. Setup dan Install Dependencies

In [ ]:
# Install dependencies
!pip install pyspark==3.5.0 pandas numpy nltk matplotlib seaborn plotly requests -q

# Download NLTK data
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('vader_lexicon', quiet=True)

print("Dependencies installed successfully!")

In [ ]:
# Import libraries
import os
import re
import json
import time
import requests
from datetime import datetime, timedelta

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window

print("Libraries imported successfully!")

In [ ]:
# Initialize Spark Session
spark = SparkSession.builder \
    .appName("StackOverflow-Analytics") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

print(f"Spark Version: {spark.version}")
print("Spark Session created successfully!")

## 2. Fetch Data dari Stack Exchange API

Mengambil data langsung dari Stack Overflow menggunakan Stack Exchange API.

**Catatan:** 
- API limit: 300 requests/day tanpa API key
- Data yang diambil: Questions dengan tags python, javascript, java
- Periode: 30 hari terakhir

In [ ]:
# Stack Exchange API Configuration
BASE_URL = "https://api.stackexchange.com/2.3"
SITE = "stackoverflow"

def fetch_questions(tag, page=1, pagesize=100):
    """Fetch questions from Stack Overflow API."""
    url = f"{BASE_URL}/questions"
    params = {
        "order": "desc",
        "sort": "votes",
        "tagged": tag,
        "site": SITE,
        "page": page,
        "pagesize": pagesize,
        "filter": "withbody"  # Include body in response
    }
    
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return None

def fetch_multiple_tags(tags, pages_per_tag=3):
    """Fetch questions for multiple tags."""
    all_questions = []
    
    for tag in tags:
        print(f"Fetching questions for tag: {tag}")
        for page in range(1, pages_per_tag + 1):
            result = fetch_questions(tag, page=page)
            if result and "items" in result:
                all_questions.extend(result["items"])
                print(f"  Page {page}: {len(result['items'])} questions")
            time.sleep(0.5)  # Rate limiting
    
    print(f"\nTotal questions fetched: {len(all_questions)}")
    return all_questions

# Fetch data untuk beberapa tags populer
tags = ["python", "javascript", "java", "pyspark", "machine-learning"]
questions_data = fetch_multiple_tags(tags, pages_per_tag=2)

In [ ]:
# Preview data structure
if questions_data:
    sample = questions_data[0]
    print("Sample Question Structure:")
    print(json.dumps({k: type(v).__name__ for k, v in sample.items()}, indent=2))
    print(f"\nSample Title: {sample.get('title', 'N/A')[:80]}...")
    print(f"Score: {sample.get('score', 0)}")
    print(f"View Count: {sample.get('view_count', 0)}")
    print(f"Tags: {sample.get('tags', [])}")

## 3. ETL Pipeline - Transform ke Spark DataFrame

Transformasi data JSON dari API ke Spark DataFrame dengan schema yang terstruktur.

In [ ]:
# Transform API data to structured format
def clean_html(text):
    """Remove HTML tags from text."""
    if text:
        clean = re.sub(r'<[^>]+>', '', text)
        return clean.strip()
    return ""

# Extract relevant fields from API response
processed_data = []
for q in questions_data:
    processed_data.append({
        "question_id": q.get("question_id"),
        "title": q.get("title", ""),
        "body": clean_html(q.get("body", "")),
        "tags": q.get("tags", []),
        "tags_str": "<" + "><".join(q.get("tags", [])) + ">" if q.get("tags") else "",
        "score": q.get("score", 0),
        "view_count": q.get("view_count", 0),
        "answer_count": q.get("answer_count", 0),
        "is_answered": 1 if q.get("is_answered", False) else 0,
        "creation_date": datetime.fromtimestamp(q.get("creation_date", 0)).strftime("%Y-%m-%d"),
        "owner_reputation": q.get("owner", {}).get("reputation", 0),
        "owner_user_id": q.get("owner", {}).get("user_id", 0),
    })

print(f"Processed {len(processed_data)} questions")
print("\nSample processed data:")
print(json.dumps(processed_data[0], indent=2, default=str))

In [ ]:
# Ensure Spark Session is active (recreate if stopped)
try:
    spark.sparkContext._jsc
except:
    print("SparkSession not active. Recreating...")
    spark = SparkSession.builder \
        .appName("StackOverflow-Analytics") \
        .config("spark.driver.memory", "4g") \
        .config("spark.sql.adaptive.enabled", "true") \
        .getOrCreate()
    spark.sparkContext.setLogLevel("ERROR")
    print("SparkSession recreated successfully!")

# Create Spark DataFrame
schema = StructType([
    StructField("question_id", IntegerType(), True),
    StructField("title", StringType(), True),
    StructField("body", StringType(), True),
    StructField("tags", ArrayType(StringType()), True),
    StructField("tags_str", StringType(), True),
    StructField("score", IntegerType(), True),
    StructField("view_count", IntegerType(), True),
    StructField("answer_count", IntegerType(), True),
    StructField("is_answered", IntegerType(), True),
    StructField("creation_date", StringType(), True),
    StructField("owner_reputation", IntegerType(), True),
    StructField("owner_user_id", IntegerType(), True),
])

# Convert to Spark DataFrame
df = spark.createDataFrame(processed_data, schema)

# Additional transformations
df = df.withColumn("creation_date", F.to_date("creation_date")) \
       .withColumn("year", F.year("creation_date")) \
       .withColumn("month", F.month("creation_date")) \
       .withColumn("title_length", F.length("title")) \
       .withColumn("body_length", F.length("body")) \
       .withColumn("tag_count", F.size("tags"))

# Handle missing values (Preprocessing)
df = df.fillna({
    "score": 0,
    "view_count": 0,
    "answer_count": 0,
    "owner_reputation": 0,
    "title": "",
    "body": ""
})

print(f"DataFrame created with {df.count()} rows")
df.printSchema()

In [ ]:
# Save to Parquet (File System requirement)
output_path = "stackoverflow_questions_parquet"
df.write.mode("overwrite").partitionBy("year", "month").parquet(output_path)
print(f"Data saved to Parquet: {output_path}")

# Show sample data
df.select("question_id", "title", "score", "view_count", "tag_count").show(10, truncate=50)

## 4. EDA - Exploratory Data Analysis

Analisis eksplorasi data untuk memahami karakteristik dataset.

In [ ]:
# Summary Statistics
print("=" * 60)
print("SUMMARY STATISTICS - STACK OVERFLOW QUESTIONS")
print("=" * 60)

stats = df.select(
    F.count("*").alias("Total Questions"),
    F.avg("score").alias("Avg Score"),
    F.avg("view_count").alias("Avg Views"),
    F.avg("answer_count").alias("Avg Answers"),
    F.sum("is_answered").alias("Answered Questions"),
    F.avg("tag_count").alias("Avg Tags"),
    F.avg("title_length").alias("Avg Title Length"),
    F.avg("body_length").alias("Avg Body Length"),
).collect()[0]

print(f"Total Questions: {stats['Total Questions']}")
print(f"Average Score: {stats['Avg Score']:.2f}")
print(f"Average Views: {stats['Avg Views']:.0f}")
print(f"Average Answers: {stats['Avg Answers']:.2f}")
print(f"Answered Questions: {stats['Answered Questions']}")
print(f"Average Tags per Question: {stats['Avg Tags']:.2f}")
print(f"Average Title Length: {stats['Avg Title Length']:.0f} chars")
print(f"Average Body Length: {stats['Avg Body Length']:.0f} chars")

In [ ]:
# Visualizations
pdf = df.toPandas()

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Score Distribution
axes[0, 0].hist(pdf['score'], bins=30, color='steelblue', edgecolor='black')
axes[0, 0].set_title('Score Distribution')
axes[0, 0].set_xlabel('Score')
axes[0, 0].set_ylabel('Frequency')

# 2. Top Tags
tag_counts = pdf['tags'].explode().value_counts().head(10)
axes[0, 1].barh(tag_counts.index, tag_counts.values, color='coral')
axes[0, 1].set_title('Top 10 Tags')
axes[0, 1].set_xlabel('Count')
axes[0, 1].invert_yaxis()

# 3. Views vs Score
axes[1, 0].scatter(pdf['view_count'], pdf['score'], alpha=0.5, c='green')
axes[1, 0].set_title('Views vs Score')
axes[1, 0].set_xlabel('View Count')
axes[1, 0].set_ylabel('Score')

# 4. Answered vs Unanswered
answered_counts = pdf['is_answered'].value_counts()
labels = ['Answered' if x == 1 else 'Unanswered' for x in answered_counts.index]
axes[1, 1].pie(answered_counts.values, labels=labels, autopct='%1.1f%%', colors=['#2ecc71', '#e74c3c'])
axes[1, 1].set_title('Answered vs Unanswered')

plt.tight_layout()
plt.show()

## 5. Spark SQL - CTE, Subquery, dan Hint

Demonstrasi penggunaan fitur Spark SQL yang advanced.

In [ ]:
# Register DataFrame as temp view
df.createOrReplaceTempView("questions")

print("=" * 70)
print("SPARK SQL - CTE, SUBQUERY, DAN SQL HINT")
print("=" * 70)

# ============================================================
# QUERY 1: CTE + Subquery + BROADCAST Hint
# ============================================================
sql_query_1 = """
-- CTE untuk filter high score questions
WITH high_score AS (
    SELECT question_id, title, score, view_count, answer_count
    FROM questions
    WHERE score >= 5
),
-- CTE untuk statistik per bulan
monthly_stats AS (
    SELECT 
        year, month,
        COUNT(*) as question_count,
        AVG(score) as avg_score,
        AVG(view_count) as avg_views
    FROM questions
    GROUP BY year, month
)
-- Query utama dengan BROADCAST hint dan subquery
SELECT /*+ BROADCAST(monthly_stats) */
    h.question_id,
    h.title,
    h.score,
    h.view_count,
    ROUND(m.avg_score, 2) as monthly_avg_score
FROM high_score h
CROSS JOIN monthly_stats m
WHERE h.score > (SELECT AVG(score) FROM questions)
ORDER BY h.score DESC
LIMIT 10
"""

print("\n📊 Query 1: CTE + Subquery + BROADCAST Hint")
print("-" * 70)
result_1 = spark.sql(sql_query_1)
display(result_1.toPandas())

# ============================================================
# QUERY 2: COALESCE Hint untuk optimasi partisi
# ============================================================
sql_query_2 = """
SELECT /*+ COALESCE(2) */
    year,
    month,
    COUNT(*) as total_questions,
    ROUND(AVG(score), 2) as avg_score,
    ROUND(AVG(view_count), 0) as avg_views,
    SUM(answer_count) as total_answers,
    SUM(CASE WHEN is_answered = 1 THEN 1 ELSE 0 END) as answered_count
FROM questions
GROUP BY year, month
ORDER BY year DESC, month DESC
"""

print("\n📊 Query 2: Monthly Statistics dengan COALESCE Hint")
print("-" * 70)
result_2 = spark.sql(sql_query_2)
display(result_2.toPandas())

# ============================================================
# QUERY 3: REPARTITION Hint + Window Function
# ============================================================
sql_query_3 = """
SELECT /*+ REPARTITION(4) */
    question_id,
    title,
    score,
    view_count,
    RANK() OVER (ORDER BY score DESC) as score_rank,
    DENSE_RANK() OVER (ORDER BY view_count DESC) as views_rank,
    ROUND(PERCENT_RANK() OVER (ORDER BY score), 4) as score_percentile
FROM questions
ORDER BY score DESC
LIMIT 10
"""

print("\n📊 Query 3: Window Functions dengan REPARTITION Hint")
print("-" * 70)
result_3 = spark.sql(sql_query_3)
display(result_3.toPandas())

## 6. RDD Operations - MapReduce

Demonstrasi operasi RDD: map, flatMap, reduceByKey, groupByKey, combineByKey, aggregateByKey, partitionBy

In [ ]:
# ============================================================
# RDD OPERATIONS - MAP, FLATMAP, REDUCEBYKEY
# ============================================================

print("=" * 70)
print("RDD OPERATIONS - MAPREDUCE")
print("=" * 70)

# Get RDD from DataFrame
rdd = df.select("question_id", "tags_str", "score", "view_count").rdd

def extract_tags(tags_str):
    """Extract individual tags from tag string."""
    if tags_str:
        return re.findall(r'<([^>]+)>', tags_str)
    return []

# ============================================================
# 1. MAP + FLATMAP + REDUCEBYKEY - Tag Count
# ============================================================
print("\n📊 1. MAP + FLATMAP + REDUCEBYKEY - Tag Count")
print("-" * 70)

# FlatMap untuk extract tags, Map untuk create pairs, ReduceByKey untuk count
tag_counts_rdd = (
    df.select("tags_str")
    .rdd
    .flatMap(lambda row: extract_tags(row["tags_str"]))  # flatMap
    .map(lambda tag: (tag, 1))  # map
    .reduceByKey(lambda a, b: a + b)  # reduceByKey
    .sortBy(lambda x: -x[1])
)

# Convert to Pandas DataFrame for table display
tag_counts_list = tag_counts_rdd.take(15)
tag_counts_df = pd.DataFrame(tag_counts_list, columns=["Tag", "Count"])
tag_counts_df.index = range(1, len(tag_counts_df) + 1)
tag_counts_df.index.name = "Rank"
display(tag_counts_df)

In [ ]:
# ============================================================
# 2. GROUPBYKEY - Questions per Tag
# ============================================================
print("\n📊 2. GROUPBYKEY - Questions per Tag")
print("-" * 70)

# GroupByKey untuk mengelompokkan question_id per tag
questions_by_tag_rdd = (
    df.select("question_id", "tags_str")
    .rdd
    .flatMap(lambda row: [(tag, row["question_id"]) for tag in extract_tags(row["tags_str"])])
    .groupByKey()  # groupByKey
    .map(lambda x: (x[0], list(x[1]), len(list(x[1]))))
    .sortBy(lambda x: -x[2])
)

# Convert to Pandas DataFrame for table display
groupby_list = [(tag, count) for tag, qids, count in questions_by_tag_rdd.take(15)]
groupby_df = pd.DataFrame(groupby_list, columns=["Tag", "Question Count"])
groupby_df.index = range(1, len(groupby_df) + 1)
groupby_df.index.name = "Rank"
display(groupby_df)

In [ ]:
# ============================================================
# 3. COMBINEBYKEY - Tag Statistics (Min, Max, Sum, Count, Avg)
# ============================================================
print("\n📊 3. COMBINEBYKEY - Tag Statistics")
print("-" * 70)

# CombineByKey untuk menghitung statistik per tag
def create_combiner(score):
    """Create initial combiner: (min, max, sum, count)"""
    return (score, score, score, 1)

def merge_value(acc, score):
    """Merge a new value into accumulator."""
    return (min(acc[0], score), max(acc[1], score), acc[2] + score, acc[3] + 1)

def merge_combiners(acc1, acc2):
    """Merge two accumulators."""
    return (min(acc1[0], acc2[0]), max(acc1[1], acc2[1]), acc1[2] + acc2[2], acc1[3] + acc2[3])

tag_stats_rdd = (
    df.select("tags_str", "score")
    .rdd
    .flatMap(lambda row: [(tag, row["score"]) for tag in extract_tags(row["tags_str"])])
    .combineByKey(create_combiner, merge_value, merge_combiners)  # combineByKey
    .map(lambda x: (x[0], x[1][0], x[1][1], x[1][2], x[1][3], round(x[1][2]/x[1][3], 2)))
    .sortBy(lambda x: -x[5])  # Sort by average
)

# Convert to Pandas DataFrame for table display
combinebykey_list = tag_stats_rdd.take(15)
combinebykey_df = pd.DataFrame(
    combinebykey_list, 
    columns=["Tag", "Min Score", "Max Score", "Total Score", "Count", "Avg Score"]
)
combinebykey_df.index = range(1, len(combinebykey_df) + 1)
combinebykey_df.index.name = "Rank"
display(combinebykey_df)

In [ ]:
# ============================================================
# 4. AGGREGATEBYKEY - Average Score per Tag
# ============================================================
print("\n📊 4. AGGREGATEBYKEY - Average Score per Tag")
print("-" * 70)

# AggregateByKey untuk menghitung rata-rata skor per tag
avg_score_rdd = (
    df.select("tags_str", "score")
    .rdd
    .flatMap(lambda row: [(tag, row["score"]) for tag in extract_tags(row["tags_str"])])
    .aggregateByKey(
        (0, 0),  # Initial value: (sum, count)
        lambda acc, score: (acc[0] + score, acc[1] + 1),  # Merge value
        lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])  # Merge combiners
    )
    .map(lambda x: (x[0], x[1][0], x[1][1], round(x[1][0] / x[1][1], 2) if x[1][1] > 0 else 0))
    .sortBy(lambda x: -x[3])
)

# Convert to Pandas DataFrame for table display
aggregatebykey_list = avg_score_rdd.take(15)
aggregatebykey_df = pd.DataFrame(
    aggregatebykey_list, 
    columns=["Tag", "Total Score", "Count", "Average Score"]
)
aggregatebykey_df.index = range(1, len(aggregatebykey_df) + 1)
aggregatebykey_df.index.name = "Rank"
display(aggregatebykey_df)

In [ ]:
# ============================================================
# 5. PARTITIONBY - Custom Partitioning
# ============================================================
print("\n📊 5. PARTITIONBY - Custom Partitioning")
print("-" * 70)

# PartitionBy untuk custom partitioning
partitioned_rdd = (
    df.select("tags_str", "score")
    .rdd
    .flatMap(lambda row: [(tag, row["score"]) for tag in extract_tags(row["tags_str"])])
    .partitionBy(4)  # partitionBy with 4 partitions
    .reduceByKey(lambda a, b: a + b)
)

print(f"✅ Number of partitions: {partitioned_rdd.getNumPartitions()}")

# Convert to Pandas DataFrame for table display
partition_list = partitioned_rdd.sortBy(lambda x: -x[1]).take(15)
partition_df = pd.DataFrame(partition_list, columns=["Tag", "Total Score"])
partition_df.index = range(1, len(partition_df) + 1)
partition_df.index.name = "Rank"
display(partition_df)

# ============================================================
# SUMMARY TABLE - All RDD Operations
# ============================================================
print("\n" + "=" * 70)
print("📋 SUMMARY: RDD Operations Used")
print("=" * 70)

summary_data = [
    ["map", "Transform each element", "tag -> (tag, 1)"],
    ["flatMap", "One-to-many transformation", "tags_str -> [tag1, tag2, ...]"],
    ["reduceByKey", "Combine values by key", "(tag, 1) -> (tag, count)"],
    ["groupByKey", "Group all values by key", "(tag, qid) -> (tag, [qid1, qid2, ...])"],
    ["combineByKey", "Custom aggregation by key", "(tag, score) -> (tag, min, max, sum, count)"],
    ["aggregateByKey", "Aggregate with initial value", "(tag, score) -> (tag, sum, count, avg)"],
    ["partitionBy", "Custom partitioning", "Distribute data across 4 partitions"],
]
summary_df = pd.DataFrame(summary_data, columns=["Operation", "Description", "Example"])
summary_df.index = range(1, len(summary_df) + 1)
display(summary_df)

## 7. NLP - Text Preprocessing

Preprocessing teks menggunakan NLTK: tokenization, stopword removal, lemmatization.

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Initialize NLTK components
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    """Preprocess text: lowercase, tokenize, remove stopwords, lemmatize."""
    if not text:
        return []
    
    # Lowercase and remove special characters
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Tokenize
    tokens = word_tokenize(text)
    
    # Remove stopwords and lemmatize
    tokens = [lemmatizer.lemmatize(token) for token in tokens 
              if token not in stop_words and len(token) > 2]
    
    return tokens

# Create UDF for Spark
@F.udf(ArrayType(StringType()))
def preprocess_udf(text):
    return preprocess_text(text)

# Apply preprocessing
df_nlp = df.withColumn("title_tokens", preprocess_udf(F.col("title"))) \
           .withColumn("body_tokens", preprocess_udf(F.col("body"))) \
           .withColumn("all_tokens", F.concat("title_tokens", "body_tokens"))

print("Text Preprocessing Applied:")
df_nlp.select("title", "title_tokens").show(5, truncate=50)

## 8. Sentiment Analysis

Analisis sentimen pada pertanyaan menggunakan VADER.

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Initialize VADER
sia = SentimentIntensityAnalyzer()

def get_sentiment(text):
    """Get sentiment score and label."""
    if not text:
        return (0.0, "neutral")
    scores = sia.polarity_scores(text)
    compound = scores['compound']
    if compound >= 0.05:
        return (compound, "positive")
    elif compound <= -0.05:
        return (compound, "negative")
    else:
        return (compound, "neutral")

# Create UDFs
@F.udf(FloatType())
def sentiment_score_udf(text):
    return float(get_sentiment(text)[0])

@F.udf(StringType())
def sentiment_label_udf(text):
    return get_sentiment(text)[1]

# Apply sentiment analysis
df_sentiment = df.withColumn("sentiment_score", sentiment_score_udf(F.col("title"))) \
                 .withColumn("sentiment_label", sentiment_label_udf(F.col("title")))

# Show results
print("Sentiment Analysis Results:")
df_sentiment.select("title", "sentiment_score", "sentiment_label").show(10, truncate=50)

# Sentiment distribution
print("\nSentiment Distribution:")
df_sentiment.groupBy("sentiment_label").count().show()

## 9. TF-IDF Feature Extraction

Ekstraksi fitur TF-IDF menggunakan Spark MLlib.

In [ ]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover, VectorAssembler

# Prepare text for TF-IDF
df_tfidf = df.withColumn("text", F.concat_ws(" ", F.col("title"), F.col("body")))

# Tokenizer
tokenizer = Tokenizer(inputCol="text", outputCol="words")
df_tokenized = tokenizer.transform(df_tfidf)

# Remove stopwords
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
df_filtered = remover.transform(df_tokenized)

# HashingTF
hashingTF = HashingTF(inputCol="filtered_words", outputCol="raw_features", numFeatures=1000)
df_tf = hashingTF.transform(df_filtered)

# IDF
idf = IDF(inputCol="raw_features", outputCol="tfidf_features")
idf_model = idf.fit(df_tf)
df_tfidf_final = idf_model.transform(df_tf)

print("TF-IDF Features Extracted:")
df_tfidf_final.select("question_id", "title", "tfidf_features").show(5, truncate=50)

## 10. Topic Modeling dengan LDA

Menggunakan Latent Dirichlet Allocation untuk menemukan topik tersembunyi.

In [ ]:
from pyspark.ml.clustering import LDA
from pyspark.ml.feature import CountVectorizer

# Prepare data for LDA
cv = CountVectorizer(inputCol="filtered_words", outputCol="cv_features", vocabSize=1000, minDF=2.0)
cv_model = cv.fit(df_filtered)
df_cv = cv_model.transform(df_filtered)

# Train LDA model
num_topics = 5
lda = LDA(k=num_topics, maxIter=10, featuresCol="cv_features")
lda_model = lda.fit(df_cv)

# Get topics
vocab = cv_model.vocabulary
topics = lda_model.describeTopics(maxTermsPerTopic=8)

print("=" * 60)
print("TOPIC MODELING RESULTS (LDA)")
print("=" * 60)

topics_list = topics.collect()
for i, row in enumerate(topics_list):
    print(f"\nTopic {i + 1}:")
    terms = [vocab[idx] for idx in row['termIndices']]
    weights = row['termWeights']
    for term, weight in zip(terms, weights):
        print(f"  - {term}: {weight:.4f}")

## 11. Machine Learning - Komparasi 2 Algoritma

Membandingkan **Random Forest** dan **Logistic Regression** untuk prediksi kualitas pertanyaan.

In [ ]:
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

# Prepare ML DataFrame
# Label: 1 = High quality (score >= 10), 0 = Low quality
df_ml = df_tfidf_final.withColumn(
    "label",
    F.when(F.col("score") >= 10, 1.0).otherwise(0.0)
)

# Assemble features
assembler = VectorAssembler(
    inputCols=["tfidf_features", "view_count", "answer_count", "tag_count"],
    outputCol="features",
    handleInvalid="skip"
)

df_ml = assembler.transform(df_ml)

print("Label Distribution:")
df_ml.groupBy("label").count().show()

# Train-test split
train_df, test_df = df_ml.randomSplit([0.7, 0.3], seed=42)
print(f"Training set: {train_df.count()} rows")
print(f"Test set: {test_df.count()} rows")

In [ ]:
# ============================================================
# MODEL 1: RANDOM FOREST with Hyperparameter Tuning
# ============================================================

print("=" * 60)
print("TRAINING RANDOM FOREST with CrossValidator")
print("=" * 60)

rf = RandomForestClassifier(featuresCol="features", labelCol="label", seed=42)

# Hyperparameter Grid
param_grid_rf = ParamGridBuilder() \
    .addGrid(rf.numTrees, [20, 50]) \
    .addGrid(rf.maxDepth, [5, 10]) \
    .build()

# Evaluator
evaluator_f1 = MulticlassClassificationEvaluator(metricName="f1")

# CrossValidator
cv_rf = CrossValidator(
    estimator=rf,
    estimatorParamMaps=param_grid_rf,
    evaluator=evaluator_f1,
    numFolds=3,
    seed=42
)

# Train
print("Training Random Forest...")
cv_model_rf = cv_rf.fit(train_df)
best_rf = cv_model_rf.bestModel

print(f"Best numTrees: {best_rf.getNumTrees}")
print(f"Best maxDepth: {best_rf.getMaxDepth()}")

# Predict
rf_predictions = cv_model_rf.transform(test_df)

In [ ]:
# ============================================================
# MODEL 2: LOGISTIC REGRESSION with Hyperparameter Tuning
# ============================================================

print("=" * 60)
print("TRAINING LOGISTIC REGRESSION with CrossValidator")
print("=" * 60)

lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=100)

# Hyperparameter Grid
param_grid_lr = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01, 0.1]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5]) \
    .build()

# CrossValidator
cv_lr = CrossValidator(
    estimator=lr,
    estimatorParamMaps=param_grid_lr,
    evaluator=evaluator_f1,
    numFolds=3,
    seed=42
)

# Train
print("Training Logistic Regression...")
cv_model_lr = cv_lr.fit(train_df)
best_lr = cv_model_lr.bestModel

print(f"Best regParam: {best_lr.getRegParam()}")
print(f"Best elasticNetParam: {best_lr.getElasticNetParam()}")

# Predict
lr_predictions = cv_model_lr.transform(test_df)

## 12. Evaluasi Model

Evaluasi kedua model dengan metrik: **Accuracy, F1-Score, Precision, Recall, AUC-ROC**

In [ ]:
# ============================================================
# MODEL EVALUATION
# ============================================================

# Evaluators
eval_accuracy = MulticlassClassificationEvaluator(metricName="accuracy")
eval_f1 = MulticlassClassificationEvaluator(metricName="f1")
eval_precision = MulticlassClassificationEvaluator(metricName="weightedPrecision")
eval_recall = MulticlassClassificationEvaluator(metricName="weightedRecall")
eval_auc = BinaryClassificationEvaluator(metricName="areaUnderROC")

# Evaluate Random Forest
rf_accuracy = eval_accuracy.evaluate(rf_predictions)
rf_f1 = eval_f1.evaluate(rf_predictions)
rf_precision = eval_precision.evaluate(rf_predictions)
rf_recall = eval_recall.evaluate(rf_predictions)
rf_auc = eval_auc.evaluate(rf_predictions)

# Evaluate Logistic Regression
lr_accuracy = eval_accuracy.evaluate(lr_predictions)
lr_f1 = eval_f1.evaluate(lr_predictions)
lr_precision = eval_precision.evaluate(lr_predictions)
lr_recall = eval_recall.evaluate(lr_predictions)
lr_auc = eval_auc.evaluate(lr_predictions)

# Print comparison
print("=" * 70)
print("MODEL COMPARISON RESULTS")
print("=" * 70)
print(f"{'Metric':<18} {'Random Forest':>20} {'Logistic Regression':>22}")
print("-" * 70)
print(f"{'Accuracy':<18} {rf_accuracy:>20.4f} {lr_accuracy:>22.4f}")
print(f"{'F1-Score':<18} {rf_f1:>20.4f} {lr_f1:>22.4f}")
print(f"{'Precision':<18} {rf_precision:>20.4f} {lr_precision:>22.4f}")
print(f"{'Recall':<18} {rf_recall:>20.4f} {lr_recall:>22.4f}")
print(f"{'AUC-ROC':<18} {rf_auc:>20.4f} {lr_auc:>22.4f}")
print("=" * 70)

# Determine best model
best_model_name = "Random Forest" if rf_f1 >= lr_f1 else "Logistic Regression"
print(f"\nBest Model (by F1-Score): {best_model_name}")

In [ ]:
# Visualization of Model Comparison
metrics = ['Accuracy', 'F1-Score', 'Precision', 'Recall', 'AUC-ROC']
rf_scores = [rf_accuracy, rf_f1, rf_precision, rf_recall, rf_auc]
lr_scores = [lr_accuracy, lr_f1, lr_precision, lr_recall, lr_auc]

x = np.arange(len(metrics))
width = 0.35

fig, ax = plt.subplots(figsize=(12, 6))
bars1 = ax.bar(x - width/2, rf_scores, width, label='Random Forest', color='steelblue')
bars2 = ax.bar(x + width/2, lr_scores, width, label='Logistic Regression', color='coral')

ax.set_xlabel('Metrics')
ax.set_ylabel('Score')
ax.set_title('Model Comparison: Random Forest vs Logistic Regression')
ax.set_xticks(x)
ax.set_xticklabels(metrics)
ax.legend()
ax.set_ylim(0, 1.1)

# Add value labels
for bar in bars1:
    height = bar.get_height()
    ax.annotate(f'{height:.3f}', xy=(bar.get_x() + bar.get_width()/2, height),
                xytext=(0, 3), textcoords="offset points", ha='center', va='bottom', fontsize=8)
for bar in bars2:
    height = bar.get_height()
    ax.annotate(f'{height:.3f}', xy=(bar.get_x() + bar.get_width()/2, height),
                xytext=(0, 3), textcoords="offset points", ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.show()

## 13. Kesimpulan

### Ringkasan Analisis Stack Overflow Data

In [ ]:
print("=" * 70)
print("RINGKASAN KEPATUHAN PERSYARATAN UAS")
print("=" * 70)

compliance_summary = """
1. DATASET 5V (min 3):
   - Volume: Data dari Stack Overflow API (ribuan pertanyaan)
   - Variety: JSON (terstruktur), teks (tidak terstruktur), tags
   - Veracity: Cleaning HTML, handling missing values
   - Value: Model prediktif kualitas pertanyaan
   
2. FILE SYSTEM: Parquet storage (dapat digunakan dengan HDFS)

3. BATCH PROCESSING + MAPREDUCE:
   - Operasi RDD: map, flatMap, reduceByKey, groupByKey
   - combineByKey, aggregateByKey, partitionBy
   
4. EDA + VISUALISASI: Matplotlib charts, statistik deskriptif

5. PREPROCESSING: Casting tipe, fillna, cleaning HTML, tokenization

6. SPARK SQL: CTE, subquery, BROADCAST hint

7. RDD OPERATIONS: Semua operasi byKey terpenuhi

8. ML KOMPARASI 2 ALGORITMA:
   - Random Forest Classifier
   - Logistic Regression
   
9. HYPERPARAMETER TUNING:
   - CrossValidator + ParamGridBuilder
   
10. EVALUASI MODEL:
    - Accuracy, F1-Score, Precision, Recall, AUC-ROC
"""

print(compliance_summary)
print("=" * 70)
print("SEMUA PERSYARATAN UAS TERPENUHI!")
print("=" * 70)

In [ ]:
# Stop Spark Session
spark.stop()
print("Spark Session stopped. Analysis complete!")